In [1]:
from hdmf.common import ExternalResources
from hdmf.common import DynamicTable, VectorData


from pynert import TermSet

import numpy as np
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb import get_type_map as tm
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject
from pynwb.behavior import SpatialSeries, Position
from datetime import datetime
from dateutil import tz

## Validate Data with a TermSet

In [2]:
terms = TermSet(name='Species_TermSet', term_schema_path='/Users/mavaylon/Research/NWB/term_set.yaml')

In [3]:
col1 = VectorData(
    name='species',
    description='...',
    data=['Homo sapiens', 'Mus musculus', 'Rattus norvegicus'],
    term_set=terms,
    validate=True
)
species = DynamicTable(name='species', description='My species', columns=[col1],)

In [4]:
col1.data # notice that the data stored is not the raw data, but the data that was confirmed to be in the termset

['Homo sapiens', 'Mus musculus']

In [5]:
col1.missing_data # you can access the missing data as such

['Rattus norvegicus']

In [6]:
col1.raw_data

['Homo sapiens', 'Mus musculus', 'Rattus norvegicus']

## Setup NWB file

In [7]:
session_start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz("US/Pacific"))

nwbfile = NWBFile(
    session_description="Mouse exploring an open field",  # required
    identifier="Mouse5_Day3",  # required
    session_start_time=session_start_time,  # required
    session_id="session_1234",  # optional
    experimenter=["Dichter, Benjamin K.", "Smith, Alex"],  # optional
    lab="My Lab Name",  # optional
    institution="University of My Institution",  # optional
    related_publications="DOI:10.1016/j.neuron.2016.12.011",  # optional
)

In [8]:
nwbfile.subject = Subject(
    subject_id="001",
    age="P90D",
    description="mouse 5",
    species="Mus musculus",
    sex="M",
)

In [9]:
species = DynamicTable(name='species', description='My species')
species.add_column(name='species', description="Name of species")
species.add_row(id=0, species='Homo sapiens')
species.add_row(id=0, species='Mus musculus')
species.add_row(id=0, species='Rattus norvegicus')

In [10]:
nwbfile.add_acquisition(species)

In [11]:
nwbfile.object_id

'8064dd07-3e62-4aed-bdfc-926575552a81'

## Add ExternalResources using a TermSet

In [12]:
er = ExternalResources(name=nwbfile.identifier, type_map=tm()) 
# using the nwbfile.identifier for the ER name to track of the corresponding nwbfile using this instance of ER

/Users/mavaylon/Research/NWB/hdmf/src/hdmf/container.py:189: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


In [13]:
terms = TermSet(name='Species_TermSet', term_schema_path='/Users/mavaylon/Research/NWB/term_set.yaml')

In [14]:
er.add_ref_term_set(object_container=nwbfile,
                    container=species,
                    attribute='species',
                    term_set=terms,
                    resource_name='NCBI_TAX'
                   ) 
# This method will return the terms that were not added to ER due to the term being missing in the termset

{'Missing Values in TermSet': ['Rattus norvegicus']}

In [15]:
er.to_dataframe()

,objects_idx,object_container_id,object_id,relative_path,field,keys_idx,key,resources_idx,resource,resource_uri,entities_idx,entity_id,entity_uri
0,0,8064dd07-3e62-4aed-bdfc-926575552a81,91569345-6bdf-475b-a4ac-44419adc3687,,,0,Homo sapiens,0,NCBI_TAX,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...,0,NCBI_TAX:9606,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
1,0,8064dd07-3e62-4aed-bdfc-926575552a81,91569345-6bdf-475b-a4ac-44419adc3687,,,1,Mus musculus,0,NCBI_TAX,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...,1,NCBI_TAX:10090,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...


## Add ER regularly

In [16]:
er = ExternalResources(name=nwbfile.identifier, type_map=tm()) 

/Users/mavaylon/Research/NWB/hdmf/src/hdmf/container.py:189: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


In [17]:
er.add_ref(
    container=nwbfile.subject,
    attribute='species',
    key='Mus musculus',
    resource_name='NCBI_Taxonomy',
    resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy',
    entity_id='NCBI:txid10090',
    entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=10090'
)

er.add_ref(
    container=nwbfile,
    attribute="experimenter",
    key="Dichter, Benjamin K.",
    resource_name="ORCID",
    resource_uri="https://orcid.org/",
    entity_id="0000-0001-5725-6910",
    entity_uri="https://orcid.org/0000-0001-5725-6910",
)


(<hdmf.common.resources.Key at 0x11dd8b910>,
 <hdmf.common.resources.Entity at 0x11dd8b400>)

## Write NWB file and ER separately 

In [18]:
io = NWBHDF5IO("NWBfile_ER_Example.nwb", mode="w")
io.write(nwbfile)
io.close()

In [19]:
# write as flat table
er.to_flat_tsv(path='./er_example_for_nwbfile.tsv')

In [20]:
#write as multple tsv files
er.to_norm_tsv(path='./')

## Read ER from tsv files of each table

In [21]:
er_read=er.from_norm_tsv(path='./')

/Users/mavaylon/Research/NWB/hdmf/src/hdmf/container.py:189: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))
/Users/mavaylon/Research/NWB/hdmf/src/hdmf/container.py:189: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


## Read ER from flat tsv

In [22]:
er_read=er.from_flat_tsv(path='./er_example_for_nwbfile.tsv')

/Users/mavaylon/Research/NWB/hdmf/src/hdmf/container.py:189: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


### Retrieve resources associated with an object

In [23]:
# Let's retrieve all external references associated with the Subject species
er_read.get_object_resources(object_container=nwbfile, container=nwbfile.subject, relative_path='Subject/species')

,keys_idx,resource_idx,entity_id,entity_uri
0,0,0,NCBI:txid10090,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
